In [180]:
#import packages
import pandas as pd
import os
import numpy as np
import sys
import math

#retrieve race keys
%store -r races
%store -r races2

In [181]:
###########################################################################################################################
#CREATE_DATA: LOAD IN DATASET
###########################################################################################################################
# INPUT: 
#      path: path csv containing block data
#      age_buckets: value for number of items in each age bucket

#  -reads in csv file to pandas dataframe
#  -replaces race string with equivalent ints 0-63 race if applicable
#  -replaces ages with appropriate age bucket labels if applicable

# OUTPUT: dataframe containing dataset with modified age and race values
##########################################################################################################################

def create_data(path, age_groupsize, max_age):
    #import original data
    df_orig = pd.read_csv (path)
    df_orig = df_orig.loc[:, ~df_orig.columns.str.contains('^Unnamed')]
    
    #get size of buckets based upon group size (if groupsize of 5, how many group of 5 buckets up to max age?)
    bucketsize = math.ceil(max_age/age_groupsize)
    
    #adjust values of necessary
    for index, row in df_orig.iterrows():    
        
        #RACE
        d = row['race']
        h = row['hispanic']
        #replace race w/ value 0-63
        if d in races:
            df_orig.at[index, 'race'] = races.index(d)
        elif d in races2:
            df_orig.at[index, 'race'] = races2.index(d)        
            
        #AGE
        #if age buckets are implemented:
        if age_groupsize > 1: 
            a = row['age']
            #get number of times groupsize goes into age:
            #ex. get new value for 12 with a group size of 5:
            #    12/5 = 2 w/remainder of 3 --> would return 2
            #set value in original dataset to this age bucket value
            df_orig.at[index, 'age'] = np.floor(a/age_groupsize)
                 
    return df_orig

In [182]:
###########################################################################################################################
#CREATE ATTRIBUTE COMBOS TO LOOK FOR IN DATASETS
###########################################################################################################################
#INPUT:
#     age_range: all possible values for age
#     race_range: all possible values for race
#     size_range: all possible values for household_size

#  -create all possible combos of age, race, household size

#OUTPUT:
#  -M: list containing all age, race, household size combos
###########################################################################################################################

def create_combos(age_range, race_range, size_range):
    
    #list containing all age, race, household size combos
    M = []
    
    #create all combos; append to M
    for age in age_range:
            for race in race_range:
                for size in size_range:
                    M.append([age, race, size])
                    
    return M

In [183]:
###########################################################################################################################
#FIND UNIQUE ROWS
###########################################################################################################################
#INPUT:
#     M: set of attributes shared between D and P ['age', 'race', 'sex']
#     D: dataframe to search for unique rows
#     theta_d: attribute set size limit in D. we only examine attribute tuples with at most theta_d matching entries in D

#  -get how many times each attribute combination occurs in the dataframe
#  -if only occurs once, add id to unique list; if occurs under the given theta_d threshold, add id to under_threshold list
#  -return these lists of rows that are unique or under threshold 

#OUTPUT:
#  -unique: list of ids of rows that are unique in the dataset
#  -under_threshold: list of ids of rows that are under the given occurence threshold in the dataset
###########################################################################################################################
    
def get_unique_rows(M, D, theta_d):

    #vals to be returned:
    #1. unique rows in D 
    unique = []
    #2. those under theta_d threshold
    under_threshold = []
   
    #for attribute combination in M
    for row in M:   
        
        #true case - get how many times that the attribute combination occurs in the deidentified dataset 
        att_val_combos = D.loc[(D['age'] == row[0]) & (D['race'] == row[1]) & (D['household_size'] == row[2])]
        
        #if combo occurs at least once and less than threshold in deidentified dataset
        if len(att_val_combos) <= theta_d and len(att_val_combos) > 0:
            #add IDs to under_threshold list
            for index, row in att_val_combos.iterrows():
                under_threshold.append([row['id'], [row['age'], row['race'], row['household_size']]])
            
        if len(att_val_combos) == 1:
            #add IDs to unique list
            for index, row in att_val_combos.iterrows():
                unique.append([row['id'], [row['age'], row['race'], row['household_size']]])
    
    return unique, under_threshold

In [184]:
###########################################################################################################################
#FIND ROW MATCHES
###########################################################################################################################
#INPUT:
#     public_dataset: publically published dataset
#     P_unique: ids of unique rows in public dataset
#     de_id_dataset: deidentified dataset
#     D_unique: ids of unique rows in de-identified dataset

#  -find all unique rows with matching row values
#  -return ids of matches

#OUTPUT:
#     matches: list of ids of all matching unique values [id in public, id in de_id]
###########################################################################################################################
def find_matches(P_unique, D_unique):
    
    #create list of unique row values only
    D_unique_rows = []
    for d in D_unique:
        D_unique_rows.append(d[1])
        
    #LIST OF ROWS WITH MATCHES
    matches = []
    
    #for each row in P_unique, see if there's a matching row in D_unique
    for p_row in P_unique:
        #see if values of [age, race, sex] associated with unique row in p (p_row) are also a unique row in D
        if p_row[1] in D_unique_rows:
            # if so, find associated ID
            for d_row in D_unique:
                if d_row[1] == p_row[1]:
                    matches.append([p_row[0], d_row[0]])
                    
    return matches

In [185]:
###########################################################################################################################
#CHECK VALIDITY OF MATCHES
###########################################################################################################################
#INPUT:
#     public_dataset: publically published dataset
#     de_id_dataset: deidentified dataset

#  -for each match, check if var predicted is the same, and if vars are the same

#OUTPUT:
#     correct_matches: matches with same id (meaning the same person's data)
#     correct_values: all reported matches that had same values (not necessarily an exact ID match)
###########################################################################################################################


def check_true_vals(matches, public_dataset, de_id_dataset):
    correct_matches = []
    correct_values = []
    
    #for each match
    for match in matches:
        
        #get values for p_id and d_id
        p_id = match[0]
        d_id = match[1]
        
        #get row values are all the same for p_id row and d_id row
        p_row = public_dataset.loc[public_dataset['id'] == d_id]
        d_row = de_id_dataset.loc[de_id_dataset['id'] == p_id]
        #check if all relevant column values match for p_id and d_id rows
        if ((p_row.iloc[0]['hispanic']==d_row.iloc[0]['hispanic'])):
            #if all values match, add to correct_values
            correct_values.append(match)
        if p_row.iloc[0]['id']==d_row.iloc[0]['id']:
            #if ids match, add to correct_matches
            correct_matches.append(match)   
    
    return correct_matches, correct_values

In [189]:

def evaluate_privacy(attribute_ranges, original_data_path, de_id_data_directory, age_groupsize, theta_d, k1, k2):
    
    correct = []
    total = []
    
    #create list of all combinations of attributes that could be present in either the public or de-identified dataset
    attribute_combos = create_combos(attribute_ranges[0], attribute_ranges[1], attribute_ranges[2])

    #create public dataset
    public_dataset = create_data(original_data_path, age_groupsize, len(attribute_ranges[0])-1)
        
    #find unique rows in public dataset
    P_unique, P_under_threshold = get_unique_rows(attribute_combos, public_dataset, theta_d)
    
    #these are the swap rates to run privacy analysis on
    swaprates = np.arange(k1, k2, .01, float).tolist()
    
    #for each de-id in given directory:
    for swaprate in swaprates:
        
        correct_matches_all =[]
        total_matches_all = []
        
        #for all de-id files in directory:
        for filename in os.listdir(de_id_data_directory):
            
            #check if file is a match for the given swaprate:
            swap_file = filename[(filename.find("_"))+1:filename.rfind("_")]
            
            #if match:
            if filename.endswith(".csv") and math.isclose(float(swaprate), float(swap_file)) and "_0.csv" in filename:
                
                #create deid-ed dataset 
                de_id_dataset = create_data(de_id_data_directory+filename, age_groupsize, len(attribute_ranges[0])-1)
                
                #find unique rows in de-ided
                D_unique, D_under_threshold = get_unique_rows(attribute_combos, de_id_dataset, theta_d)

                #find matches for unique rows
                matches = find_matches(P_unique, D_unique)

                #use ID to check true values (predict var & see if var is correct)
                correct_matches, total_matches = check_true_vals(matches, public_dataset, de_id_dataset)

                correct_matches_all.append(correct_matches)
                total_matches_all.append(total_matches)
        
        correct_matches_total = [0,0]
        total_matches_total = [0,0]

        for s in correct_matches_all:
            correct_matches_total[0] = correct_matches_total[0] + len(s)
            correct_matches_total[1] = correct_matches_total[1] + 1

        for s in total_matches_all:
            total_matches_total[0] = total_matches_total[0] + len(s)
            total_matches_total[1] = total_matches_total[1] + 1

        corr_matches = correct_matches_total[0]/correct_matches_total[1]
        tot_matches = total_matches_total[0]/total_matches_total[1]

        correct.append([swaprate, corr_matches])
        total.append([swaprate, tot_matches])
        
        print([swaprate, corr_matches], [swaprate, tot_matches])
    
    return correct, total
    

In [ ]:
#RUNNER FUNCTION

def runner_function(county, swapping_directory_type, age_groupsize, theta_d, start_rate, stop_rate):
    attribute_ranges = [range(90), range(len(races)), [1,2,3,4]]
    correct, total = evaluate_privacy(attribute_ranges, '../homemade_data/' + county + '.csv', 
                 '../swapping/swap_runs/'+county+'/'+swapping_directory_type+'/',
                 age_groupsize, theta_d, start_rate, stop_rate)
    with open(county+"_"+swapping_directory_type+"_"+age_groupsize+'_privacy.txt', "a") as f:
        f.write("Correct:\n\n")
        for c in correct:
            f.write(c + "\n")
        f.write("Total:\n\n")
        for t in total:
            f.write(t + "\n")
                     
                
runner_function('alameda', 'similar_0', 15, 3, 0.01, 1.0)

[0.01, 82.0] [0.01, 82.0]
[0.02, 80.0] [0.02, 80.0]
[0.03, 76.0] [0.03, 76.0]
[0.04, 69.0] [0.04, 69.0]
[0.05, 73.0] [0.05, 73.0]
[0.060000000000000005, 66.0] [0.060000000000000005, 66.0]
[0.06999999999999999, 58.0] [0.06999999999999999, 59.0]
[0.08, 57.0] [0.08, 58.0]
[0.09, 52.0] [0.09, 52.0]
[0.09999999999999999, 57.0] [0.09999999999999999, 57.0]
[0.11, 53.0] [0.11, 55.0]
[0.12, 44.0] [0.12, 45.0]
[0.13, 40.0] [0.13, 41.0]
[0.14, 43.0] [0.14, 43.0]
[0.15000000000000002, 33.0] [0.15000000000000002, 33.0]
[0.16, 32.0] [0.16, 34.0]
[0.17, 31.0] [0.17, 33.0]
[0.18000000000000002, 30.0] [0.18000000000000002, 32.0]
[0.19, 23.0] [0.19, 27.0]
[0.2, 26.0] [0.2, 29.0]
[0.21000000000000002, 18.0] [0.21000000000000002, 25.0]
[0.22, 17.0] [0.22, 22.0]
[0.23, 16.0] [0.23, 23.0]
[0.24000000000000002, 17.0] [0.24000000000000002, 22.0]
[0.25, 14.0] [0.25, 18.0]
[0.26, 14.0] [0.26, 18.0]
[0.27, 13.0] [0.27, 19.0]
[0.28, 13.0] [0.28, 16.0]
[0.29000000000000004, 20.0] [0.29000000000000004, 24.0]
[0.3, 